In [1]:
%%capture
%pip install -U bitsandbytes
%pip install transformers==4.44.2
%pip install -U accelerate
%pip install -U peft
%pip install -U trl

In [2]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("HUGGINGFACE_TOKEN")
login(token = hf_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
base_model_url = "meta-llama/Llama-3.2-3B-Instruct"
new_model_url = "yuva44/llama-3.2-3b-it-Ecommerce-ChatBot"

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
from peft import PeftModel
import torch
from trl import setup_chat_format
# Reload tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(base_model_url)

base_model_reload= AutoModelForCausalLM.from_pretrained(
    base_model_url,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [5]:
# Merge adapter with base model
base_model_reload, tokenizer = setup_chat_format(base_model_reload, tokenizer)
model = PeftModel.from_pretrained(base_model_reload, new_model_url)

model = model.merge_and_unload()

adapter_config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/1.67G [00:00<?, ?B/s]

In [6]:
instruction = """You are a top-rated customer service agent named John. 
    Be polite to customers and answer all their questions.
    """

messages = [{"role": "system", "content": instruction},
    {"role": "user", "content": "I have to see what payment payment modalities are accepted"}]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    
inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

outputs = model.generate(**inputs, max_new_tokens=150, num_return_sequences=1)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text.split("assistant")[1])


Sure! I understand your need to explore the various payment methods we accept. Let me guide you through the different options available to you:

1. **Credit/Debit Card**: We accept major credit and debit cards like Visa, Mastercard, and American Express.
2. **PayPal**: You can make secure payments using your PayPal account.
3. **Bank Transfer**: If you prefer a more traditional payment method, you can make a direct bank transfer.
4. **Apple Pay**: For Apple users, we offer the convenience of Apple Pay for a seamless payment experience.
5. **Google Pay**: Android users can also enjoy the ease of making payments with Google Pay.

These are just a few of the payment methods we offer. If you have


In [7]:
from huggingface_hub import HfApi
import os
api = HfApi()

In [8]:
api.create_repo(
    repo_id=repo_id,
    token=hf_token,
    private=False,
    exist_ok=True
)

# Save model and tokenizer locally
local_path = model_name
model.save_pretrained(local_path, safe_serialization=True)
tokenizer.save_pretrained(local_path)

NameError: name 'repo_id' is not defined

In [ ]:
api.upload_folder(
    folder_path=local_path,
    repo_id=repo_id,
    repo_type="model",
    token=hf_token,
    commit_message="Upload model and tokenizer files"
)

print(f"Model successfully uploaded to https://huggingface.co/{repo_id}")